In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)

### MNIST dataset

In [ ]:
(X_train, y_train), (X_test, y_test)= keras.datasets.mnist.load_data()

X_train= X_train[:1000].reshape(-1, 28*28) / 255.0
X_test= X_test[:1000].reshape(-1, 28*28) / 255.0

y_train= y_train[:1000]
y_test= y_test[:1000]

In [ ]:
plt.imshow(X_train[24].reshape((28,28)))

### Define a model

In [ ]:
def create_model():
    model= keras.Sequential([
        keras.layers.Dense(512, activation= tf.nn.relu, input_shape= (28*28, )),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation= tf.nn.softmax)
    ])
    
    model.compile(optimizer= tf.train.AdamOptimizer(),
                 loss= tf.keras.losses.sparse_categorical_crossentropy,
                 metrics= ['accuracy'])
    
    return model

In [ ]:
model= create_model()
model.summary()

### Save checkpoint during training

In [ ]:
import os
checkpoint_path= 'training_1/cp.ckpt'
checkpoing_dir= os.path.dirname(checkpoint_path)

#### Checkpoing callback

In [ ]:
cp_callback= tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                               save_weights_only= True,
                                               verbose= 2)

In [ ]:
model= create_model()

In [ ]:
model.fit(X_train, y_train,
         epochs= 10,
         validation_data= (X_test, y_test),
         callbacks= [cp_callback])

In [ ]:
! dir {checkpoing_dir}

### Restoring on untrained model

In [ ]:
new_model= create_model()

In [ ]:
loss, acc= new_model.evaluate(X_test, y_test)
print(loss, acc*100.0)

In [ ]:
new_model.load_weights(checkpoint_path)

In [ ]:
loss, acc= new_model.evaluate(X_test, y_test)
print(loss, acc*100.0)

### Checkpoing on every 5 epochs

In [ ]:
checkpoint_path= 'training_2/cp-{epoch:04d}.ckpt'
checkpoing_dir= os.path.dirname(checkpoint_path)

In [ ]:
cp_callback= keras.callbacks.ModelCheckpoint(checkpoint_path,
                                            verbose= 1,
                                            save_weights_only= True,
                                            period= 5)

In [ ]:
model= create_model()

In [ ]:
model.fit(X_train, y_train,
         epochs= 50,
         callbacks= [cp_callback],
         validation_data= (X_test, y_test),
         verbose= 0)

In [ ]:
! dir {checkpoing_dir}

#### Getting the latest model

In [ ]:
latest= tf.train.latest_checkpoint(checkpoing_dir)
latest

In [ ]:
model= create_model()
model.load_weights(latest)
loss, acc= model.evaluate(X_test, y_test)
print(loss, acc*100.0)

### Manually save the weights

In [ ]:
#Saving
model.save_weights('./checkpoints/shiv_checkpoint')

#Restoring
model= create_model()
model.load_weights('./checkpoints/shiv_checkpoint')

loss, acc= model.evaluate(X_test, y_test)
print(loss, acc* 100.0)

### Saving the entire model

In [ ]:
model= create_model()

In [ ]:
#Need to use keras.optimizer to restore optimizer state from HDF5 file
model.compile(optimizer= keras.optimizers.Adam(),
             loss= keras.losses.sparse_categorical_crossentropy,
             metrics= ['accuracy'])

In [ ]:
model.fit(X_train, y_train,
         validation_data= (X_test, y_test),
         epochs= 5)

In [ ]:
model.save('shiv_model.h5')

In [ ]:
new_model= keras.models.load_model('shiv_model.h5')
new_model.summary()

In [ ]:
loss, acc= model.evaluate(X_test, y_test)
print(loss, acc* 100.0)

#### To be continued....